In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tools import download, time, visualize
from selenium.common.exceptions import TimeoutException
import matplotlib as mpl

## 1. Download Data 

### Load transients Catalogue

In [3]:
df_transients = time.time(
    download.all_transients_catalog,
    overwrite=False
)

0:00:00.016670


In [4]:
df_transients.head()

,clas,date,dec,ra,type
0,"SN SDSS mag 20,8",20161109.49,20.60814,142.63453,SUPERNOVAE
1,SN (ATLAS16dsr) SDSS mag,20161109.24,13.91412,57.65194,SUPERNOVAE
2,"SN (ATLAS16dtz) mag 14,5",20161108.48,55.58309,141.31900,SUPERNOVAE
3,SN SDSS mag >22,20161107.28,23.18541,21.02794,SUPERNOVAE
4,"SN (MLS161020) SDSS mag 21,4",20161106.22,20.34622,10.39148,SUPERNOVAE


In [5]:
df_transients.count()

clas    7943
date    7943
dec     7943
ra      7943
type    7943
dtype: int64

### Load Light Curves

In [6]:
df_light_curves = time.time(
    download.all_transients_light_curves,
    df_transients,
    overwrite=False
)

File all_transients_light_curves.pickle already exists
0:00:00.262216


In [7]:
df_light_curves.head()

,ObjectID,ID,Mag,Magerr,RA,Decl,MJD,Blend,Date
0,0,1121048016831,21.15,0.52,142.63264,20.60759,53739.39280,0,2006-01-04 09:25:37.920
1,0,1121048016831,20.31,0.33,142.63520,20.60753,53761.33345,0,2006-01-26 08:00:10.080
2,0,1121048016831,20.73,0.45,142.63637,20.60929,53875.18592,0,2006-05-20 04:27:43.488
3,0,1121048016831,20.37,0.32,142.63480,20.60820,54054.46708,0,2006-11-15 11:12:35.712
4,0,1121048016831,20.44,0.36,142.63371,20.60857,54197.14661,0,2007-04-07 03:31:07.104


In [8]:
df_light_curves.ID.count()

1017388

In [9]:
df_light_curves.groupby(['ObjectID']).ID.count().describe()

count    7835.000000
mean      129.851691
std       131.708264
min         1.000000
25%        27.500000
50%        83.000000
75%       195.000000
max       922.000000
Name: ID, dtype: float64

In [10]:
len(df_light_curves.ObjectID.unique())

7835

## 2. Object Catalog Statistics by Type

### Supernovae

In [11]:
df_transients_supernovae = df_transients[df_transients.type =='SUPERNOVAE']
df_transients_supernovae.head()

,clas,date,dec,ra,type
0,"SN SDSS mag 20,8",20161109.49,20.60814,142.63453,SUPERNOVAE
1,SN (ATLAS16dsr) SDSS mag,20161109.24,13.91412,57.65194,SUPERNOVAE
2,"SN (ATLAS16dtz) mag 14,5",20161108.48,55.58309,141.31900,SUPERNOVAE
3,SN SDSS mag >22,20161107.28,23.18541,21.02794,SUPERNOVAE
4,"SN (MLS161020) SDSS mag 21,4",20161106.22,20.34622,10.39148,SUPERNOVAE


In [12]:
df_transients_supernovae.count()

clas    2714
date    2714
dec     2714
ra      2714
type    2714
dtype: int64

### Cataclysmic Variable Stars

In [13]:
df_transients_cv = df_transients[df_transients.type =='CV']
df_transients_cv.head()

,clas,date,dec,ra,type
2714,"CV SDSS mag >22,5",20161109,61.47645,157.35429,CV
2715,"CV SDSS mag 21,8",20161109,0.22334,127.37297,CV
2716,CV mag 21,20161023,57.38627,99.82350,CV
2717,CV (PS1_15aaf) mag 21,20161013,57.27874,93.68036,CV
2718,"CV SDSS mag 21,2",20161012,61.40397,139.18290,CV


In [14]:
df_transients_cv.count()

clas    1362
date    1362
dec     1362
ra      1362
type    1362
dtype: int64

### Blazars

In [15]:
df_transients_blazars = df_transients[df_transients.type =='BLAZARS']
df_transients_blazars.head()

,clas,date,dec,ra,type
4076,Blazar PKS_2247-131,20161009,-12.85467,342.49855,BLAZARS
4077,"Blazar (0748+333) SDSS mag 19,6",20161007,33.22214,117.97364,BLAZARS
4078,Blazar CRATES_J0001+2113,20160709,21.22680,0.38474,BLAZARS
4079,"Blazar SDSS mag 21,3",20160708,52.82354,212.85706,BLAZARS
4080,"Blazar SDSS mag 19,6",20160616,9.97402,177.82208,BLAZARS


In [16]:
df_transients_blazars.count()

clas    396
date    396
dec     396
ra      396
type    396
dtype: int64

### AGNs

In [17]:
df_transients_agn = df_transients[df_transients.type =='AGN']
df_transients_agn.head()

,clas,date,dec,ra,type
4472,"AGN SDSS mag 21,3",20161107,19.57569,24.14416,AGN
4473,"AGN SDSS mag 22,0",20161107,0.63542,22.13552,AGN
4474,"AGN SDSS mag 20,4",20160904,-9.70794,30.53147,AGN
4475,"AGN SDSS mag 21,1",20160903,-5.47395,347.98532,AGN
4476,"AGN SDSS mag 20,1",20160710,35.99400,254.28421,AGN


In [18]:
df_transients_agn.count()

clas    3471
date    3471
dec     3471
ra      3471
type    3471
dtype: int64

## 3. Light Curves Catalog Statistics by Type

### Supernovae

In [19]:
df_light_curves_supernovae = df_light_curves[df_light_curves.ObjectID.isin(df_transients_supernovae.index)]
df_light_curves_supernovae.head()

,ObjectID,ID,Mag,Magerr,RA,Decl,MJD,Blend,Date
0,0,1121048016831,21.15,0.52,142.63264,20.60759,53739.39280,0,2006-01-04 09:25:37.920
1,0,1121048016831,20.31,0.33,142.63520,20.60753,53761.33345,0,2006-01-26 08:00:10.080
2,0,1121048016831,20.73,0.45,142.63637,20.60929,53875.18592,0,2006-05-20 04:27:43.488
3,0,1121048016831,20.37,0.32,142.63480,20.60820,54054.46708,0,2006-11-15 11:12:35.712
4,0,1121048016831,20.44,0.36,142.63371,20.60857,54197.14661,0,2007-04-07 03:31:07.104


In [20]:
df_light_curves_supernovae.count()

ObjectID    183055
ID          183055
Mag         183055
Magerr      183055
RA          183055
Decl        183055
MJD         183055
Blend       183055
Date        183055
dtype: int64

In [21]:
series_light_curves_supernovae_grouped = df_light_curves_supernovae.groupby(['ObjectID']).Mag.count()

In [22]:
series_light_curves_supernovae_grouped.describe()

count    2625.000000
mean       69.735238
std        99.108401
min         1.000000
25%        11.000000
50%        26.000000
75%        80.000000
max       671.000000
Name: Mag, dtype: float64

In [23]:
len(series_light_curves_supernovae_grouped[series_light_curves_supernovae_grouped > 10].index)

1995

### Cataclysmic Variable Stars

In [24]:
df_light_curves_cv = df_light_curves[df_light_curves.ObjectID.isin(df_transients_cv.index)]
df_light_curves_cv.head()

,ObjectID,ID,Mag,Magerr,RA,Decl,MJD,Blend,Date
183080,2714,6011116002931020219,19.070527,0.207620,157.35316,61.47558,53746.274027,0,2006-01-11 06:34:35.960016
183081,2715,11012510104641005092,20.099937,0.344394,127.37181,0.22320,55586.334373,0,2011-01-25 08:01:29.816988
183082,2715,9042010104621003854,20.954769,0.531688,127.37287,0.22300,54941.139811,0,2009-04-20 03:21:19.668016
183083,2716,13101615702032023407,20.658646,0.432848,99.82311,57.38648,56581.434008,0,2013-10-16 10:24:58.261988
183084,2716,10031815702011022420,20.589015,0.406931,99.82317,57.38622,55273.169179,0,2010-03-18 04:03:37.056986


In [25]:
df_light_curves_cv.count()

ObjectID    201025
ID          201025
Mag         201025
Magerr      201025
RA          201025
Decl        201025
MJD         201025
Blend       201025
Date        201025
dtype: int64

In [26]:
series_light_curves_cv_grouped = df_light_curves_cv.groupby(['ObjectID']).Mag.count()

In [27]:
series_light_curves_cv_grouped.describe()

count    1345.000000
mean      149.460967
std       152.521062
min         1.000000
25%        21.000000
50%        92.000000
75%       251.000000
max       904.000000
Name: Mag, dtype: float64

In [28]:
len(series_light_curves_cv_grouped[series_light_curves_cv_grouped > 10].index)

1181

### Blazars

In [29]:
df_light_curves_blazars = df_light_curves[df_light_curves.ObjectID.isin(df_transients_blazars.index)]
df_light_curves_blazars.head()

,ObjectID,ID,Mag,Magerr,RA,Decl,MJD,Blend,Date
384170,4076,12091901212031023084,19.169053,0.212302,342.49776,-12.85600,56189.270321,0,2012-09-19 06:29:15.717008
384171,4076,10100101212031019387,20.429292,0.479209,342.49803,-12.85488,55470.244969,0,2010-10-01 05:52:45.357015
384172,4076,13102501212011021450,20.777713,0.509270,342.49803,-12.85499,56590.129157,0,2013-10-25 03:05:59.205011
384173,4076,13093001212011020719,20.178060,0.347259,342.49817,-12.85484,56565.216036,0,2013-09-30 05:11:05.489983
384174,4076,13093001212021021007,19.945571,0.305662,342.49860,-12.85484,56565.221072,0,2013-09-30 05:18:20.594007


In [30]:
df_light_curves_blazars.count()

ObjectID    93660
ID          93660
Mag         93660
Magerr      93660
RA          93660
Decl        93660
MJD         93660
Blend       93660
Date        93660
dtype: int64

In [31]:
series_light_curves_blazars_grouped = df_light_curves_blazars.groupby(['ObjectID']).Mag.count()

In [32]:
series_light_curves_blazars_grouped.describe()

count    395.000000
mean     237.113924
std      160.459333
min        3.000000
25%      102.500000
50%      224.000000
75%      331.000000
max      730.000000
Name: Mag, dtype: float64

In [33]:
len(series_light_curves_blazars_grouped[series_light_curves_blazars_grouped > 10].index)

390

### AGN

In [34]:
df_light_curves_agn = df_light_curves[df_light_curves.ObjectID.isin(df_transients_agn.index.tolist())]
df_light_curves_agn = df_light_curves_agn[df_light_curves_agn.MJD > 0]
df_light_curves_agn.head()

,ObjectID,ID,Mag,Magerr,RA,Decl,MJD,Blend,Date
477867,4472,2120022002399,21.53,0.27,24.14481,19.57607,54450.24708,0,2007-12-16 05:55:47.712
477868,4472,2120022002399,22.29,0.40,24.14406,19.57609,54450.26080,0,2007-12-16 06:15:33.120
477869,4472,2120022002399,22.60,0.48,24.14416,19.57507,54450.26080,1,2007-12-16 06:15:33.120
477870,4472,2120022002399,22.22,0.39,24.14430,19.57526,54468.12235,0,2008-01-03 02:56:11.040
477871,4472,2120022002399,21.61,0.28,24.14488,19.57647,54779.10981,0,2008-11-09 02:38:07.584


In [35]:
df_light_curves_agn.count()

ObjectID    539648
ID          539648
Mag         539648
Magerr      539648
RA          539648
Decl        539648
MJD         539648
Blend       539648
Date        539648
dtype: int64

In [36]:
series_light_curves_agn_grouped = df_light_curves_agn.groupby(['ObjectID']).Mag.count()

In [37]:
series_light_curves_agn_grouped.describe()

count    3470.000000
mean      155.518156
std       122.290693
min         1.000000
25%        65.000000
50%       124.000000
75%       211.000000
max       922.000000
Name: Mag, dtype: float64

In [38]:
len(series_light_curves_agn_grouped[series_light_curves_agn_grouped > 10].index)

3448

## 4. Light Curves Visualization

In [39]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### Supernovae

#### Generate Light Curves Visualizations

##### Scatter of all observations

In [40]:
visualize.light_curve(df_light_curves_supernovae, filename='supernovae')

##### Grouped subset of observations per season

In [41]:
visualize.combined_light_curves(df_light_curves_supernovae, n_obj=15, obj_type='supernovae', min_obs=15)

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13952.0, right=13952.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=12940.0, right=12940.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=14274.0, right=14274.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13230.0, right=13230.0

/Users/diego/Documents/astro/venv/lib/python3.4/

#### Individual supernovae

In [42]:
index_supernovae_light_curve = 10
# Get sample
df_light_curves_supernovae_sample = df_light_curves_supernovae[
    df_light_curves_supernovae.ObjectID == df_light_curves_supernovae.ObjectID.unique()[index_supernovae_light_curve]
]

In [43]:
iplot([visualize.light_curve_interactive_scatter(df_light_curves_supernovae_sample)])

### Cataclysmic Variable Stars

#### Generate Light Curves Visualizations

##### Scatter of all observations

In [44]:
visualize.light_curve(df_light_curves_cv, filename='cv')

##### Grouped subset of observations per season

In [45]:
visualize.combined_light_curves(df_light_curves_cv, n_obj=15, obj_type='cv', min_obs=15)

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=12943.0, right=12943.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13048.0, right=13048.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=12945.0, right=12945.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13554.0, right=13554.0

/Users/diego/Documents/astro/venv/lib/python3.4/

#### Individual CVs

In [46]:
index_cv_light_curve = 10
# Get sample
df_light_curves_cv_sample = df_light_curves_cv[
    df_light_curves_cv.ObjectID == df_light_curves_cv.ObjectID.unique()[index_cv_light_curve]
]

In [47]:
iplot([visualize.light_curve_interactive_scatter(df_light_curves_cv_sample)])

### Blazars

#### Generate Light Curves Visualizations

##### Scatter of all observations

In [48]:
visualize.light_curve(df_light_curves_blazars, filename='blazars')

##### Grouped subset of observations per season

In [49]:
visualize.combined_light_curves(df_light_curves_blazars, n_obj=15, obj_type='blazars', min_obs=15)

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13057.0, right=13057.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13270.0, right=13270.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13591.0, right=13591.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13493.0, right=13493.0

/Users/diego/Documents/astro/venv/lib/python3.4/

#### Individual blazars

In [50]:
index_blazars_light_curve = 10
# Get sample
df_light_curves_blazars_sample = df_light_curves_blazars[
    df_light_curves_blazars.ObjectID == df_light_curves_blazars.ObjectID.unique()[index_blazars_light_curve]
]

In [51]:
iplot([visualize.light_curve_interactive_scatter(df_light_curves_blazars_sample)])

### AGNs

#### Generate Light Curves Visualizations

##### Scatter of all observations

In [52]:
visualize.light_curve(df_light_curves_agn, filename='agn')

##### Grouped subset of observations per season

In [53]:
visualize.combined_light_curves(df_light_curves_agn, n_obj=15, obj_type='agn', min_obs=15)

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=12882.0, right=12882.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=12937.0, right=12937.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=12907.0, right=12907.0

/Users/diego/Documents/astro/venv/lib/python3.4/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=13084.0, right=13084.0

/Users/diego/Documents/astro/venv/lib/python3.4/

#### Individual AGNs

In [54]:
index_agn_light_curve = 10
# Get sample
df_light_curves_agn_sample = df_light_curves_agn[
    df_light_curves_agn.ObjectID == df_light_curves_agn.ObjectID.unique()[index_agn_light_curve]
]

In [55]:
iplot([visualize.light_curve_interactive_scatter(df_light_curves_agn_sample)])